<a href="https://colab.research.google.com/github/EchenimEdwin/https-github.com-users-EchenimEdwin-/blob/main/SCADA_System_Python_Backend_(Flask).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# scada_backend.py
%pip install flask flask-cors
import time
import random
from collections import deque
from flask import Flask, jsonify, request
from flask_cors import CORS # To allow cross-origin requests from the frontend

app = Flask(__name__)
CORS(app) # Enable CORS for all routes

# --- Configuration Parameters ---
# Sensor data limits
TEMP_MIN, TEMP_MAX = 20.0, 30.0
PRESSURE_MIN, PRESSURE_MAX = 50.0, 70.0
FLOW_MIN, FLOW_MAX = 100.0, 150.0

# Alarm thresholds
TEMP_ALARM_HIGH = 28.0
PRESSURE_ALARM_HIGH = 65.0
FLOW_ALARM_LOW = 110.0

# Data history for AI/ML (e.g., last 10 readings)
HISTORY_LENGTH = 10

# --- Global Data Storage ---
# Using a dictionary for history to make it accessible outside the loop
global_sensor_data_history = {
    'temperature': deque(maxlen=HISTORY_LENGTH),
    'pressure': deque(maxlen=HISTORY_LENGTH),
    'flow': deque(maxlen=HISTORY_LENGTH),
}
global_current_alarms = set() # Stores active alarm messages
global_last_data = {} # To store the last generated data

# --- 1. Data Generation Function ---
def generate_sensor_data():
    """Simulates real-time sensor data with some fluctuations."""
    temp = round(random.uniform(TEMP_MIN, TEMP_MAX) + random.gauss(0, 1), 2)
    pressure = round(random.uniform(PRESSURE_MIN, PRESSURE_MAX) + random.gauss(0, 0.5), 2)
    flow = round(random.uniform(FLOW_MIN, FLOW_MAX) + random.gauss(0, 2), 2)

    # Introduce occasional "faults" for testing alarm/AI
    if random.random() < 0.15: # Increased chance of a fault for demonstration
        fault_type = random.choice(['temp_spike', 'pressure_drop', 'flow_surge', 'temp_drop', 'pressure_spike', 'flow_drop'])
        if fault_type == 'temp_spike':
            temp = round(random.uniform(TEMP_ALARM_HIGH + 1, TEMP_MAX + 5), 2)
        elif fault_type == 'pressure_drop':
            pressure = round(random.uniform(PRESSURE_MIN - 5, PRESSURE_ALARM_HIGH - 1), 2)
        elif fault_type == 'flow_surge':
            flow = round(random.uniform(FLOW_MIN - 10, FLOW_ALARM_LOW - 1), 2) # Simulate low flow as a surge
        elif fault_type == 'temp_drop':
            temp = round(random.uniform(TEMP_MIN - 5, TEMP_MIN + 1), 2)
        elif fault_type == 'pressure_spike':
            pressure = round(random.uniform(PRESSURE_ALARM_HIGH + 1, PRESSURE_MAX + 5), 2)
        elif fault_type == 'flow_drop':
            flow = round(random.uniform(FLOW_ALARM_LOW - 5, FLOW_ALARM_LOW - 1), 2)

    return {'temperature': temp, 'pressure': pressure, 'flow': flow}

# --- 3. Alarm Triggering Function ---
def check_alarms(data):
    """
    Checks sensor data against predefined thresholds and updates active alarms.
    Updates global_current_alarms.
    """
    new_alarms = set()
    # Using specific strings for alarms to allow for easier removal
    temp_alarm_msg = f"High Temperature: {data['temperature']:.2f}°C (Threshold: >{TEMP_ALARM_HIGH}°C)"
    pressure_alarm_msg = f"High Pressure: {data['pressure']:.2f} PSI (Threshold: >{PRESSURE_ALARM_HIGH} PSI)"
    flow_alarm_msg = f"Low Flow Rate: {data['flow']:.2f} L/s (Threshold: <{FLOW_ALARM_LOW} L/s)"

    if data['temperature'] > TEMP_ALARM_HIGH:
        new_alarms.add(temp_alarm_msg)
    else:
        global_current_alarms.discard(temp_alarm_msg)

    if data['pressure'] > PRESSURE_ALARM_HIGH:
        new_alarms.add(pressure_alarm_msg)
    else:
        global_current_alarms.discard(pressure_alarm_msg)

    if data['flow'] < FLOW_ALARM_LOW:
        new_alarms.add(flow_alarm_msg)
    else:
        global_current_alarms.discard(flow_alarm_msg)

    # Update global alarms set
    global_current_alarms.update(new_alarms)

# --- 4. AI/ML Model Integration (Simulated) ---
def analyze_with_ai_ml(data_history):
    """
    Simulates an AI/ML model analyzing historical and current data.
    Provides more varied insights based on trends and patterns.
    """
    insight = "System operating normally. No significant patterns detected by AI."

    # Ensure enough data for analysis
    if any(len(history) < HISTORY_LENGTH for history in data_history.values()):
        return "AI is gathering more data for comprehensive analysis..."

    # Convert deques to lists for easier slicing and numerical operations
    temp_hist = list(data_history['temperature'])
    pressure_hist = list(data_history['pressure'])
    flow_hist = list(data_history['flow'])

    latest_temp = temp_hist[-1]
    latest_pressure = pressure_hist[-1]
    latest_flow = flow_hist[-1]

    # --- Simulated Anomaly Detection (using recent average and deviation) ---
    # Focus on the last few readings for 'sudden' changes
    recent_window = 3 # Look at last 3 readings
    if len(temp_hist) >= recent_window:
        avg_temp_recent = sum(temp_hist[-recent_window:]) / recent_window
        std_dev_temp = (sum([(x - avg_temp_recent)**2 for x in temp_hist[-recent_window:]]) / recent_window)**0.5
        if abs(latest_temp - avg_temp_recent) > 2 * std_dev_temp and std_dev_temp > 0.5: # If current deviates significantly from recent average
            return f"AI Anomaly Alert: Temperature ({latest_temp:.2f}°C) is deviating significantly from recent average. Possible sensor drift or rapid change detected!"

    if len(pressure_hist) >= recent_window:
        avg_pressure_recent = sum(pressure_hist[-recent_window:]) / recent_window
        std_dev_pressure = (sum([(x - avg_pressure_recent)**2 for x in pressure_hist[-recent_window:]]) / recent_window)**0.5
        if abs(latest_pressure - avg_pressure_recent) > 2 * std_dev_pressure and std_dev_pressure > 0.2:
            return f"AI Anomaly Alert: Pressure ({latest_pressure:.2f} PSI) is showing unusual fluctuations. Investigate valve stability."

    if len(flow_hist) >= recent_window:
        avg_flow_recent = sum(flow_hist[-recent_window:]) / recent_window
        std_dev_flow = (sum([(x - avg_flow_recent)**2 for x in flow_hist[-recent_window:]]) / recent_window)**0.5
        if abs(latest_flow - avg_flow_recent) > 2 * std_dev_flow and std_dev_flow > 1.0:
            return f"AI Anomaly Alert: Flow rate ({latest_flow:.2f} L/s) is highly erratic. Potential blockage or pump issue suspected."

    # --- Simulated Predictive Maintenance (trends over entire history) ---
    # Trend analysis for temperature
    temp_trend = temp_hist[-1] - temp_hist[0] # Difference between latest and oldest
    if temp_trend > 4.0:
        return f"AI Predictive: Sustained increasing temperature trend ({temp_trend:.2f}°C rise over {HISTORY_LENGTH} readings). Recommend thermal system check soon."
    elif temp_trend < -4.0:
        return f"AI Predictive: Sustained decreasing temperature trend ({abs(temp_trend):.2f}°C drop). Could indicate cooling system over-performance or sensor issue."

    # Trend analysis for pressure
    pressure_trend = pressure_hist[-1] - pressure_hist[0]
    if pressure_trend > 3.0:
        return f"AI Predictive: Pressure showing a consistent upward trend ({pressure_trend:.2f} PSI rise). Monitor for potential over-pressurization."
    elif pressure_trend < -3.0:
        return f"AI Predictive: Pressure is consistently dropping ({abs(pressure_trend):.2f} PSI drop). Check for leaks or pump efficiency."

    # Trend analysis for flow
    flow_trend = flow_hist[-1] - flow_hist[0]
    if flow_trend < -8.0:
        return f"AI Predictive: Significant downward trend in flow rate ({abs(flow_trend):.2f} L/s drop). Suggests impending blockage or pump degradation."
    elif flow_trend > 8.0:
        return f"AI Predictive: Flow rate consistently increasing ({flow_trend:.2f} L/s rise). Check for unintended bypass or control system issues."


    # --- Simulated Root Cause Analysis (combining conditions) ---
    if "High Temperature" in global_current_alarms and "High Pressure" in global_current_alarms:
        return "AI Root Cause Hint: Both high temp and high pressure detected. This often points to a cooling system failure or closed valve."
    if "Low Flow Rate" in global_current_alarms and (latest_pressure < PRESSURE_MIN + 2):
        return "AI Root Cause Hint: Low flow combined with low pressure suggests a supply issue or major leak before the sensor."
    if "Low Flow Rate" in global_current_alarms and (latest_pressure > PRESSURE_ALARM_HIGH - 2):
        return "AI Root Cause Hint: Low flow with normal/high pressure could indicate a blockage after the sensor or pump issue."

    # If all else, provide a general status
    return insight

# --- Flask API Endpoints ---
@app.route('/api/status', methods=['GET'])
def get_scada_status():
    """
    Returns the current sensor data, active alarms, and AI insight.
    This also advances the simulation by one step.
    """
    global global_last_data
    global global_sensor_data_history
    global global_current_alarms

    # 1. Generate new data
    new_data = generate_sensor_data()
    global_last_data = new_data # Store for consistent display

    # Update data history for AI/ML
    for sensor, value in new_data.items():
        global_sensor_data_history[sensor].append(value)

    # 2. Check for alarms
    check_alarms(new_data)

    # 3. Get AI/ML insight
    ai_insight = analyze_with_ai_ml(global_sensor_data_history)

    response = {
        'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
        'sensor_data': new_data,
        'active_alarms': sorted(list(global_current_alarms)), # Convert set to sorted list for consistent order
        'ai_insight': ai_insight
    }
    return jsonify(response)

@app.route('/api/reset', methods=['POST'])
def reset_scada_system():
    """Resets the SCADA simulation state."""
    global global_sensor_data_history
    global global_current_alarms
    global global_last_data

    for key in global_sensor_data_history:
        global_sensor_data_history[key].clear()
    global_current_alarms.clear()
    global_last_data = {} # Clear last data as well

    return jsonify({"message": "SCADA system reset successfully."})

if __name__ == '__main__':
    # This will run the Flask app. For development, use debug=True
    # For deployment, use a production-ready WSGI server like Gunicorn or Waitress
    print("Starting Flask SCADA Backend. Access at http://127.0.0.1:5000")
    app.run(debug=True, port=5000)


Starting Flask SCADA Backend. Access at http://127.0.0.1:5000
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
